In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

tqdm.pandas()

In [2]:
## Run the following to compute the mappings for host locations
#!python3 -m scripts.mappings_host_location

## Load Visualization Pipeline for all the periods
## Insert one of the following arguments to run the visualization:
## - sep
## - dic
## - mar
## - jun

#!python3 -m scripts.pipeline_viz sep
#!python3 -m scripts.pipeline_viz dic
#!python3 -m scripts.pipeline_viz mar
#!python3 -m scripts.pipeline_viz jun

#!python3 -m scripts.pipeline_ML sep
#!python3 -m scripts.pipeline_ML dic
#!python3 -m scripts.pipeline_ML mar
#!python3 -m scripts.pipeline_ML jun